In [4]:
import os
import sys
import numpy as np

base_path = '../../../'
sys.path.append(os.path.join(os.getcwd(), base_path))

dataset_path = 'datasets/numpy/pokemon100ms_no_vel_transposed/meta/'
dataset_file = '_dicted_dataset_ignore_ratio=0.1.npy'
word_vectors_file = '_word_vectors_1000_ignore_ratio=0.1.wv'

track_path = os.path.join(base_path, dataset_path, dataset_file)
word_vectors_path = os.path.join(base_path, dataset_path, word_vectors_file)

track = np.load(track_path)

track

array(['D#3,C4', 'D#3,C4', 'D#3,C4', ..., '<UNKNOWN>', '<UNKNOWN>',
       '<TRACK_END>'], dtype='<U27')

In [17]:
from src.data_processing.embedding_sparse_notes.common import TRACK_END, UNKNOWN_FRAME

track_split_points = np.where(track == TRACK_END)[0]

# + 1, so split happens after <TRACK_END>, [:-1] to skip last, empty partition
tracks = [t.tolist() for t in np.split(track, track_split_points + 1)][:-1]
# tracks is now a list of lists of frames

In [22]:
from gensim.models import KeyedVectors

wv = KeyedVectors.load(word_vectors_path, mmap='r')

def vectorize_frame(frame):
    return wv[frame] if frame in wv else wv[UNKNOWN_FRAME]

vectorized_tracks = [np.array([vectorize_frame(f) for f in t]) for t in tracks]
for v in vectorized_tracks: print(v.shape, end=', ')

(2197, 16), (2197, 16), (1920, 16), (361, 16) , (552, 16) , (1008, 16), (533, 16) , (552, 16) , (552, 16) , (533, 16) , (1008, 16), (361, 16) , (2197, 16), (2197, 16), (2197, 16), (361, 16) , (1920, 16), (552, 16) , (1008, 16), (552, 16) , (717, 16) , (717, 16) , (552, 16) , (1008, 16), (552, 16) , (1920, 16), (361, 16) , (2197, 16), (2197, 16), (1920, 16), (552, 16) , (1622, 16), (717, 16) , (386, 16) , (1008, 16), (1622, 16), (552, 16) , (552, 16) , (1622, 16), (1008, 16), (386, 16) , (717, 16) , (552, 16) , (1622, 16), (1920, 16), (361, 16) , (2197, 16), (2197, 16), (1920, 16), (361, 16) , (552, 16) , (717, 16) , (1008, 16), (386, 16) , (552, 16) , (1008, 16), (1008, 16), (552, 16) , (386, 16) , (1008, 16), (1622, 16), (552, 16) , (361, 16) , (1920, 16), (2197, 16), (717, 16) , (361, 16) , (2197, 16), (1008, 16), (533, 16) , (533, 16) , (386, 16) , (386, 16) , (533, 16) , (533, 16) , (361, 16) , (2197, 16), (1008, 16), (361, 16) , (717, 16) , (717, 16) , (1622, 16), (2197, 16), (192

In [82]:
def dataset_gen(tracks, window_size, batch_size):
    """
    tracks - list of np.arrays of shape (track_length, frame_size)
    window_size - length of generated batch
    batch_size - number of sequences in batch
    """
    while True:
        # select #batch_size tracks
        selected_track_indicies = [np.random.randint(0, len(tracks)) for _ in range(batch_size)]
        # select sequence starting point for each track
        sequence_indicies = [np.random.randint(0, len(tracks[sti]) - window_size - 2)
                             for sti in selected_track_indicies]
        
        
        # create slices for x and y
        x_slice = lambda seqi: np.s_[seqi:seqi + window_size]
        y_slice = lambda seqi: np.s_[seqi + 1:seqi + window_size + 1]
        
        x = [tracks[sti][x_slice(seqi)] for sti, seqi in zip(selected_track_indicies, sequence_indicies)]
        y = [tracks[sti][y_slice(seqi)] for sti, seqi in zip(selected_track_indicies, sequence_indicies)]

        yield np.stack(x), np.stack(y)
        
x, y = next(dataset_gen(vectorized_tracks, 50, 5))
x.shape, y.shape

(50, 16)
(50, 16)
(50, 16)
(50, 16)
(50, 16)


((5, 50, 16), (5, 50, 16))